In [2]:
%matplotlib inline
import sys
import re
import nltk
import numpy as np
import pandas as pd
import pymorphy2
import sklearn
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer
from keras import regularizers

morph = pymorphy2.MorphAnalyzer()

w2v = KeyedVectors.load_word2vec_format("data/ruwikiruscorpora_0_300_20.bin", binary=True)
w2v.init_sims(replace=True)

C:\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [4]:
data = pd.read_csv('data/250ktendersList_norm.csv', sep='\t')
data.head()

,tender_id,tender_name,lots,otrasl,podortasl,okpd2,okpd2_name,norm_name,norm_lots,norm_okpd
0,50000048,Филиал Федерального бюджетного учреждения здра...,Услуги по оценке собственности,"Бизнес, финансы, страхование, маркетинг и реклама","Бухгалтерский учет, оценка, аудит",683116120,Услуги посреднические при оценке нежилого недв...,филиал_NOUN федеральный_ADJ бюджетный_ADJ учре...,услуга_NOUN оценка_NOUN собственность_NOUN,услуга_NOUN посреднический_ADJ оценка_NOUN неж...
1,50000049,МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ САРГАТСКОГ...,Поставка газа,Топливо и энергетика,Газ,352000000,"Газы горючие искусственные, услуги по распреде...",муниципальный_ADJ унитарный_ADJ предприятие_NO...,поставка_NOUN газа_NOUN,газа_NOUN горючий_ADJ искусственный_ADJ услуга...
2,50000050,АКЦИОНЕРНАЯ КОМПАНИЯ 'АЛРОСА' (ПУБЛИЧНОЕ АКЦИО...,НИГП. Разработка минералого-геохимических крит...,"Строительство, недвижимость и архитектура","Проектные работы, геодезия, картография, инжен...",711239000,"Услуги в области геологических, геофизических ...",акционерный_ADJ компания_NOUN алрос_NOUN публи...,разработка_NOUN геохимический_ADJ критерий_NOU...,услуга_NOUN область_NOUN геологический_ADJ гео...
3,50000149,АКЦИОНЕРНОЕ ОБЩЕСТВО 'ТРАНСНЕФТЬ - ЦЕНТРАЛЬНАЯ...,Техническое обслуживание теплоэнергетического ...,"Офис, дом",Услуги,353012000,Услуги по снабжению паром и горячей водой по т...,акционерный_ADJ общество_NOUN транснефть_NOUN ...,технический_ADJ обслуживание_NOUN теплоэнергет...,услуга_NOUN снабжение_NOUN паром_NOUN горячий_...
4,50000153,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ЭНЕРГЕТИКИ И ЭЛ...,Открытый запрос предложений № 476-ОЗП/16-СМП н...,"Офис, дом",Мебель,310100000,Мебель для офисов и предприятий торговли,публичный_ADJ акционерный_ADJ общество_NOUN эн...,открытый_ADJ запрос_NOUN предложение_NOUN опре...,мебель_NOUN офис_NOUN предприятие_NOUN торговл...


In [7]:
y1_encoder = sklearn.preprocessing.LabelEncoder()
y1_encoder.fit(data['otrasl'])
y1_all = y1_encoder.transform(data['otrasl'])
print(dict(enumerate(y1_encoder.classes_)))

{0: 'IT, компьютеры, связь', 1: 'Безопасность', 2: 'Бизнес, финансы, страхование, маркетинг и реклама', 3: 'Бумажное производство, тара и упаковка', 4: 'Деревообработка, лес', 5: 'Издательство, полиграфия', 6: 'Легкая промышленность', 7: 'Машиностроение', 8: 'Медицина, фармакология', 9: 'Металлы, металлоизделия', 10: 'Наука, исследования, образование', 11: 'Офис, дом', 12: 'Перевозки, логистика, таможня', 13: 'Продовольствие, пищевая промышленность', 14: 'Сельское хозяйство', 15: 'Спорт, отдых, туризм', 16: 'Строительство, недвижимость и архитектура', 17: 'Сырье, полуфабрикаты', 18: 'Топливо и энергетика', 19: 'Транспорт', 20: 'Химия', 21: 'Экология', 22: 'Электротехника'}


In [3]:
data['norm_podortasl'] = data['otrasl'] + ' - ' + data['podortasl']

def create_target(length, index):
    x = np.zeros(length)
    x[0] = 0.1
    x[index] = 1;
    return x

def create_targets(column):
    names = list(set([name for name in column]))
    names.append('0_unknown')
    names.sort()
    names_dict = dict([(v, i) for i, v in enumerate(names)])
    name_count = len(names)
    targets = np.array([create_target(name_count, names_dict[name]) for name in column])
    return targets, names

y1_all, y1_names = create_targets(data['otrasl'])
y2_all, y2_names = create_targets(data['norm_podortasl'])

print(y1_all.shape)
print(y2_all.shape)

(249778, 24)
(249778, 132)


In [113]:
for i in range(0, len(y2_names)):
    print(i, y2_names[i])

0 0_unknown
1 IT, компьютеры, связь - Web, Интернет
2 IT, компьютеры, связь - Защита информации
3 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
4 IT, компьютеры, связь - Программное обеспечение
5 IT, компьютеры, связь - Радиосвязь
6 IT, компьютеры, связь - Спутниковое телевидение
7 IT, компьютеры, связь - Телефония
8 IT, компьютеры, связь - Услуги провайдеров, хостинг, прочие услуги
9 Безопасность - Видеонаблюдение, Сигнализация, ОПС, СКД
10 Безопасность - Другое
11 Безопасность - Охрана и оборудование
12 Безопасность - Противопожарная безопасность
13 Бизнес, финансы, страхование, маркетинг и реклама - Банковское дело
14 Бизнес, финансы, страхование, маркетинг и реклама - Бухгалтерский учет, оценка, аудит
15 Бизнес, финансы, страхование, маркетинг и реклама - Реклама, PR, Маркетинг
16 Бизнес, финансы, страхование, маркетинг и реклама - Страхование
17 Бизнес, финансы, страхование, маркетинг и реклама - Сувенирная продукция
18 Бизнес, финансы,

In [114]:
# Маски для разделения выборок
np.random.seed(1)
rand_mask = np.random.rand(len(data))
split_ratio = 0.8
mask_tr = rand_mask <= split_ratio
mask_val = rand_mask > split_ratio

y1_tr = y1_all[mask_tr]
y1_val = y1_all[mask_val]
y2_tr = y2_all[mask_tr]
y2_val = y2_all[mask_val]

y_names = y2_names
y_all = y2_all
y_tr = y2_tr
y_val = y2_val

len(y_tr), len(y_val)

(199924, 49854)

In [115]:
for i in range(0, len(y_names)):
    print(i, y_names[i])

0 0_unknown
1 IT, компьютеры, связь - Web, Интернет
2 IT, компьютеры, связь - Защита информации
3 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
4 IT, компьютеры, связь - Программное обеспечение
5 IT, компьютеры, связь - Радиосвязь
6 IT, компьютеры, связь - Спутниковое телевидение
7 IT, компьютеры, связь - Телефония
8 IT, компьютеры, связь - Услуги провайдеров, хостинг, прочие услуги
9 Безопасность - Видеонаблюдение, Сигнализация, ОПС, СКД
10 Безопасность - Другое
11 Безопасность - Охрана и оборудование
12 Безопасность - Противопожарная безопасность
13 Бизнес, финансы, страхование, маркетинг и реклама - Банковское дело
14 Бизнес, финансы, страхование, маркетинг и реклама - Бухгалтерский учет, оценка, аудит
15 Бизнес, финансы, страхование, маркетинг и реклама - Реклама, PR, Маркетинг
16 Бизнес, финансы, страхование, маркетинг и реклама - Страхование
17 Бизнес, финансы, страхование, маркетинг и реклама - Сувенирная продукция
18 Бизнес, финансы,

In [116]:
for i in range(0, 3):
    y = np.argmax(y_all[i])
    print(y, y_names[y])

14 Бизнес, финансы, страхование, маркетинг и реклама - Бухгалтерский учет, оценка, аудит
103 Топливо и энергетика - Газ
92 Строительство, недвижимость и архитектура - Проектные работы, геодезия, картография, инженерные изыскания, кадастровые работы


In [134]:
words_sets = list()
for i in range(len(data)):
    if i % 100 == 0:
        sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
        sys.stdout.flush()
    words = set(str(data['norm_lots'][i]).split())
    words = words or set(str(data['norm_okpd'][i]).split())
    words = [word for word in words if word in w2v.vocab]
    words_sets.append(words)
print('. Complete.')

Process: 249700 of 249778. Complete.


In [135]:
for i in range(13, 14):
    y = np.argmax(y_all[i])
    print(y, y_names[y])
    print(words_sets[i])

3 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
['нужда_NOUN', 'овд_NOUN', 'центр_NOUN', 'омский_ADJ', 'фазан_NOUN', 'тип_NOUN', 'приобретение_NOUN', 'радиостанция_NOUN']


In [136]:
def build_words_data(mask):
    words_separated = list()
    xw = list()
    yw = list()
    print()
    for i in range(len(data)):
        if i % 100 == 0:
            sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
            sys.stdout.flush()
        if not mask[i]:
            continue
        y_current = y_all[i]
        words = words_sets[i]
        for word in words:
            words_separated.append(word)
            xw.append(w2v.word_vec(word))
            yw.append(y_current)
    xw = np.array(xw)
    yw = np.array(yw)
    print('. Complete.')
    print(xw.shape, yw.shape)
    return words_separated, xw, yw
            
words_tr, xw_tr, yw_tr = build_words_data(mask_tr)
words_val, xw_val, yw_val = build_words_data(mask_val)


Process: 249700 of 249778. Complete.
(1590870, 300) (1590870, 132)

Process: 249700 of 249778. Complete.
(397292, 300) (397292, 132)


In [137]:
for i in range(30, 40):
    y = np.argmax(yw_tr[i])
    print(y, y_names[y])
    print(words_tr[i])

62 Офис, дом - Мебель
поставка_NOUN
62 Офис, дом - Мебель
депо_NOUN
62 Офис, дом - Мебель
хороший_ADJ
62 Офис, дом - Мебель
запрос_NOUN
62 Офис, дом - Мебель
предложение_NOUN
46 Машиностроение - Ремонт, обслуживание, пуско-наладочные работы
количество_NOUN
46 Машиностроение - Ремонт, обслуживание, пуско-наладочные работы
система_NOUN
46 Машиностроение - Ремонт, обслуживание, пуско-наладочные работы
котлоагрегат_NOUN
46 Машиностроение - Ремонт, обслуживание, пуско-наладочные работы
взамен_ADV
46 Машиностроение - Ремонт, обслуживание, пуско-наладочные работы
сигнализация_NOUN


In [138]:
w_in_len = xw_tr.shape[1]
w_out_len = yw_tr.shape[1]

print(w_in_len, '->', w_out_len)

model_words_pre = Sequential()

model_words_pre.add(Dense(256, input_dim=(w_in_len)))
model_words_pre.add(Dropout(0.5))
model_words_pre.add(Activation('tanh'))

model_words_pre.add(Dense(w_out_len))
model_words_pre.add(Activation('softmax'))

model_words_pre.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

history = model_words_pre.fit(
    xw_tr, 
    yw_tr,
    batch_size=1024,
    epochs=2)

yw_pred = model_words_pre.predict(xw_val, batch_size=1024)

#model.evaluate(xw_val, yw_val, batch_size=128)

yw_true = np.argmax(yw_pred, axis=1) == np.argmax(yw_val, axis=1)
print('accuracy:', np.mean(yw_true))

300 -> 132
Epoch 1/2
1590870/1590870 [==============================] - 46s - loss: 3.8086 - acc: 0.1681    
Epoch 2/2
1590870/1590870 [==============================] - 44s - loss: 3.6910 - acc: 0.1875    
accuracy: 0.188667277468


In [139]:
for i in range(0, 1000):
    curr_val = np.argmax(yw_val[i])
    curr_pred = np.argmax(yw_pred[i])
    if curr_pred == 0:
        continue
    print(curr_val, y_names[curr_val])
    print(curr_pred, y_names[curr_pred])
    print(words_val[i])

3 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
50 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы
нужда_NOUN
3 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
64 Офис, дом - Услуги
овд_NOUN
3 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
50 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы
центр_NOUN
3 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
50 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы
тип_NOUN
3 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
8 IT, компьютеры, связь - Услуги провайдеров, хостинг, прочие услуги
радиостанция_NOUN
37 Легкая промышленность - Хозяйственные изделия
50 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы
средс

7 IT, компьютеры, связь - Телефония
связь_NOUN
7 IT, компьютеры, связь - Телефония
64 Офис, дом - Услуги
услуга_NOUN
2 IT, компьютеры, связь - Защита информации
4 IT, компьютеры, связь - Программное обеспечение
система_NOUN
2 IT, компьютеры, связь - Защита информации
4 IT, компьютеры, связь - Программное обеспечение
информация_NOUN
2 IT, компьютеры, связь - Защита информации
4 IT, компьютеры, связь - Программное обеспечение
обеспечение_NOUN
2 IT, компьютеры, связь - Защита информации
50 Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы
защита_NOUN
2 IT, компьютеры, связь - Защита информации
57 Наука, исследования, образование - Услуги
аттестация_NOUN
2 IT, компьютеры, связь - Защита информации
64 Офис, дом - Услуги
оказание_NOUN
2 IT, компьютеры, связь - Защита информации
64 Офис, дом - Услуги
услуга_NOUN
2 IT, компьютеры, связь - Защита информации
4 IT, компьютеры, связь - Программное обеспечение
автоматизированный_ADJ
57 Наука, исследования, образование 

In [140]:
model_words = Sequential()

model_words.add(Dense(256, input_dim=(w_in_len), 
    weights=model_words_pre.layers[0].get_weights()))
model_words.add(Activation('sigmoid'))


In [141]:
model_words_pre.predict(np.array([w2v.word_vec('газ_NOUN')]))

array([[  7.62861818e-02,   2.02783431e-05,   6.99808006e-05,
          3.59378150e-03,   2.03719968e-03,   1.69516337e-04,
          2.82667643e-05,   7.00106437e-04,   2.27569998e-03,
          4.64983378e-03,   1.38091500e-05,   3.93751496e-03,
          1.95522211e-03,   1.92033345e-04,   7.62335816e-03,
          4.08418709e-04,   1.33221372e-04,   4.34785943e-05,
          2.57078675e-04,   5.32565929e-04,   5.69319036e-06,
          6.85649400e-04,   9.38117300e-05,   7.72531668e-04,
          7.23051053e-05,   8.71961063e-04,   4.27442501e-06,
          7.56345325e-05,   1.60271316e-04,   3.71702306e-04,
          1.66372425e-04,   2.76394326e-06,   1.47992541e-05,
          2.16252702e-05,   1.94513894e-04,   4.78143047e-04,
          6.13651937e-05,   5.39065944e-03,   1.08552654e-03,
          1.89481289e-05,   2.12958511e-02,   1.59388539e-04,
          3.06950435e-02,   2.21092370e-03,   1.07151933e-03,
          2.98106764e-03,   1.56183140e-02,   1.43858360e-03,
        

In [142]:
texts_vectors = list()

for i in range(len(data)):
    if i % 100 == 0:
        sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
        sys.stdout.flush()
    words_vectors = [w2v.word_vec(w) for w in words_sets[i]]
    if not words_vectors:
        words_vectors.append(np.zeros(300))
    pred = model_words_pre.predict(np.array(words_vectors))
    texts_vectors.append(np.sum(pred, axis=0))
    '''
    conc_vector = np.concatenate((
        np.amax(pred, axis=0),
        np.amin(pred, axis=0)
    ))
    texts_vectors.append(conc_vector)
    '''

texts_vectors = np.array(texts_vectors)

print('. Complete.')

Process: 249700 of 249778. Complete.


In [143]:
okpd_vectors = list()

for i in range(len(data)):
    if i % 100 == 0:
        sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
        sys.stdout.flush()
    okpd = data['okpd2'][i]
    x = np.zeros(410)
    x[okpd//10000000] = 1;
    x[100 + okpd//100000%100] = 1;
    x[200 + okpd//1000%100] = 1;
    x[300 + okpd//10%100] = 1;
    x[400 + okpd%10] = 1;
    okpd_vectors.append(x)

okpd_vectors = np.array(okpd_vectors)
    
print('. Complete.')

Process: 249700 of 249778. Complete.


In [144]:
def create_x_1(index):
    return okpd_vectors[index]

def create_x_2(index):
    return np.concatenate((
        okpd_vectors[index],
        texts_vectors[index],
    ))

def create_x_3(index):
    return texts_vectors[index]

def create_x_4(index):
    return [okpd_vectors[index], texts_vectors[index]]

def create_x(index):
    if index % 100 == 0:
        sys.stdout.write('\rProcess: %d of %d' % (index, len(data)))
        sys.stdout.flush()
    return create_x_4(index)

#x_all = np.array([create_x(i) for i in range(0, len(data))])
x_all = np.array([create_x(i) for i in range(0, len(data))])

print('. Complete.')

x_tr = x_all[mask_tr]
x_val = x_all[mask_val]

Process: 249700 of 249778. Complete.


In [145]:
okpd_tr = okpd_vectors[mask_tr]
okpd_val = okpd_vectors[mask_val]
print(okpd_tr.shape, okpd_val.shape)

texts_tr = texts_vectors[mask_tr]
texts_val = texts_vectors[mask_val]
print(texts_tr.shape, texts_val.shape)

print(y_tr.shape, y_val.shape)

(199924, 410) (49854, 410)
(199924, 132) (49854, 132)
(199924, 132) (49854, 132)


In [146]:
import keras.layers

okpd_input_len = okpd_tr.shape[1]
text_input_len = texts_tr.shape[1]
out_len = y_tr.shape[1]

layer_okpd_input = Input(shape=(okpd_input_len,))

layer_okpd = Dense(512, activation='relu')(layer_okpd_input)
layer_okpd = Dropout(0.5)(layer_okpd)

layer_text_input = Input(shape=(text_input_len,))

layer_text = Dense(256, activation='tanh')(layer_text_input)
layer_text = Dropout(0.5)(layer_okpd)

layer_merged = keras.layers.Concatenate()([layer_okpd, layer_text])
layer_merged = Dense(256, activation='relu')(layer_merged)
#layer_merged = Dense(256, activation='relu')(layer_okpd)
layer_merged = Dropout(0.5)(layer_merged)

layer_output = Dense(out_len, activation='softmax')(layer_merged)

model = Model(inputs=[layer_okpd_input, layer_text_input], outputs=[layer_output])

model.compile(loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

history = model.fit([okpd_tr, texts_tr], y_tr,
        batch_size=128,
        epochs=3)

y_pred = model.predict([okpd_val, texts_val], batch_size=128)

model.evaluate([okpd_val, texts_val], y_val, batch_size=128)

Epoch 1/3
199924/199924 [==============================] - 29s - loss: 2.0724 - acc: 0.5904    
Epoch 2/3
199924/199924 [==============================] - 28s - loss: 1.5608 - acc: 0.7162    
Epoch 3/3
49408/49854 [============================>.] - ETA: 0s

[1.2875040267144806, 0.78776025997103638]

In [147]:
y_true = [int(np.argmax(y_pred[i]) == np.argmax(y_val[i])) for i in range(len(y_val))]
print('accuracy:', np.mean(y_true))

accuracy: 0.787760259959


In [148]:
y_pred_all = model.predict([okpd_vectors, texts_vectors], batch_size=128)


In [149]:
y_pred_all_argmax = np.argmax(y_pred_all, axis=1)

data['result'] = [y_names[y_pred_all_argmax[i]] for i in range(len(data))]

y_false = np.argmax(y_pred_all, axis=1) != np.argmax(y_all, axis=1)
data_false = data.loc[y_false]
data_false = data_false.drop(['norm_name', 'norm_lots', 'norm_okpd', 'norm_podortasl'], axis=1)
data_false.head()

,tender_id,tender_name,lots,otrasl,podortasl,okpd2,okpd2_name,result
2,50000050,АКЦИОНЕРНАЯ КОМПАНИЯ 'АЛРОСА' (ПУБЛИЧНОЕ АКЦИО...,НИГП. Разработка минералого-геохимических крит...,"Строительство, недвижимость и архитектура","Проектные работы, геодезия, картография, инжен...",711239000,"Услуги в области геологических, геофизических ...","Наука, исследования, образование - Услуги"
5,50000155,Филиал 'Астраханский судоремонтный завод ' АО ...,"Доработка системы автоматического управления, ...",Машиностроение,"Ремонт, обслуживание, пуско-наладочные работы",331910000,Услуги по ремонту прочего оборудования,"Медицина, фармакология - Медицинское оборудова..."
6,50000234,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ 'ОРЕН...,Поставка клапанов обратных для нужд ООО 'Оренб...,"Строительство, недвижимость и архитектура",Ремонтные и строительные работы,281411130,Клапаны обратные,"Металлы, металлоизделия - Металлоизделия и мет..."
10,50000273,АКЦИОНЕРНОЕ ОБЩЕСТВО 'ДАЛЬНЕВОСТОЧНАЯ ГЕНЕРИРУ...,Установка железнодорожных весов ВТЭЦ-2,"Строительство, недвижимость и архитектура",Ремонтные и строительные работы,331910000,Услуги по ремонту прочего оборудования,"Медицина, фармакология - Медицинское оборудова..."
16,50000497,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО 'ЭЛЕКТРОПРИВОД'...,Оказание услуг по аренде испытательного оборуд...,Транспорт,"Автомобили, автотранспорт, автобусы, прицепы",773919000,Услуги по аренде и лизингу прочих машин и обор...,"Транспорт - Транспортные услуги, перевозки"


In [150]:
data_false.to_csv('data/250ktendersList_false.csv', sep='\t', index=False, encoding='utf-8')